# EEG signals are less sensitive than LFP signals to the exact distributions of synaptic inputs

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import neuron
import LFPy
from brainsignals.plotting_convention import mark_subplots, cmap_v_e
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

ns.load_mechs_from_folder(ns.cell_models_folder)

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]
radii = np.array([89000., 90000., 95000., 100000.])  # (µm)
sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)

rad_tol = 1e-2
sigma = 0.3
num_elecs = 12
# Define electrode parameters
elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': np.zeros(num_elecs),  # electrode positions
    'y': np.zeros(num_elecs),
    'z': np.linspace(-1200, -10, num_elecs),
    'method': 'root_as_point'
}
dz = np.abs(elec_params["z"][1] - elec_params["z"][0])

synapse_params = {
    'syntype' : 'ExpSynI',      # current-based exponential synapse
    'tau' : 1.,                #Time constant, rise
    'weight' : 0.005,           #Synaptic weight
    'record_current' : False,    #record synaptic currents
}


In [ ]:
def insert_synapses(cell, synapse_params, z_min,  spiketimes):
    """ Find n compartments to insert synapses onto """
    n = len(spiketimes)
    idx = cell.get_rand_idx_area_norm(section="allsec", nidx=n, z_min=z_min)
    for i in idx:
        synapse_params.update({'idx' : int(i)})
        s = LFPy.Synapse(cell, **synapse_params)
        s.set_spike_times(np.array([spiketimes[i]]))


def plot_four_sphere_model(ax, radii):
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='k', lw=.1))
    ax.add_patch(Ellipse((0, radii[-1]), 2000, 500, color='gray'))


def plot_laminar_lfp(lfp, ax, tvec, normalize):
    z = elec_params["z"]
    dz = np.abs(z[1] - z[0])
    lfp_ = lfp / normalize
    for elec in range(lfp.shape[0]):
        ax.plot(tvec, lfp_[elec] * dz / 1.5 + z[elec], c='k', lw=1)
    img = ax.imshow(lfp, cmap=cmap_v_e, origin="lower",
                    vmax=normalize, vmin=-normalize,
                    extent=[0, tvec[-1], np.min(z) - dz/2, np.max(z) + dz/2])
    ax.axis("auto")
    return img

In [ ]:
num_syns = 1000
input_t_center = 10
input_t_std = 2
tstop = 40
dt = 2**-4
tvec = np.arange(tstop / dt + 1) * dt

z_mins = np.array([-600, -400, -200])

num_trials = 10
seeds = np.random.randint(1, 124121, num_trials)
lfp_dict = {}
eeg_dict = {}

for i, z_min in enumerate(z_mins):
    eeg_trial = []
    lfp_trial = []

    for trial, seed in enumerate(seeds):
        np.random.seed(seed)
        spiketimes = np.random.normal(input_t_center, input_t_std, size=num_syns)
        cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
        cell.set_pos(z=-np.max(cell.z) - 10)
        insert_synapses(cell, synapse_params, z_min, spiketimes)
        cell.simulate(rec_imem=True, rec_vmem=True)

        electrode = LFPy.RecExtElectrode(cell, **elec_params)
        electrode.LFP = electrode.get_transformation_matrix() @ cell.imem

        somapos = np.array([0., 0., radii[0] + cell.z[0].mean()])
        r_soma_syns = [cell.get_intersegment_vector(idx0=0, idx1=i)
                       for i in cell.synidx]
        r_mid = np.average(r_soma_syns, axis=0)
        r_mid = somapos + r_mid/2.

        eeg_coords_top = np.array([[0., 0., radii[-1] - rad_tol]])

        four_sphere_top = LFPy.FourSphereVolumeConductor(eeg_coords_top,
                                                         radii, sigmas)
        cdm = LFPy.CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem

        pot_db_4s_top = four_sphere_top.get_transformation_matrix(r_mid) @ cdm

        eeg_trial.append(np.array(pot_db_4s_top)[0] * 1e6)
        lfp_trial.append(electrode.LFP * 1000)

        del cell
        del electrode
    eeg_dict[z_min] = np.sum(eeg_trial, axis=0)
    lfp_dict[z_min] = np.sum(lfp_trial, axis=0)



In [ ]:
depth_clrs = {z_min: plt.cm.rainbow(i / (len(z_mins) - 1))
              for i, z_min in enumerate(z_mins)}

ax_h_eeg = 0.1
ax_h_lfp = 0.6
ax_lfp_hstart = 0.05
ax_eeg_hstart = 0.72
ax_w = 0.215
ax_wspace = 0.01
ax_left = 0.0

plt.close("all")
fig = plt.figure()

ax_grid_lfp = [ax_left, 0, ax_w, 0.66]
ax_morph = fig.add_axes(ax_grid_lfp, frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[-350, 350],
                        ylim=[-1400, 100])

ax_grid_4s = [0.01, ax_eeg_hstart, 0.2, 0.2]
ax_4s = fig.add_axes(ax_grid_4s, frameon=False, aspect=1,
                     xticks=[], yticks=[],
                     xlim=[-17000, 17000],
                     ylim=[87000, 102000])
ax_4s.set_title("head model")
plot_four_sphere_model(ax_4s, radii)

ax_morph.plot(elec_params["x"], elec_params["z"], 'o',
              c='cyan', ms=4, zorder=1)
ax_morph.axhline(0, c='gray', ls="--")
ax_morph.text(-400, 20, "z=0 µm", va="bottom")

cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
cell.set_pos(z=-np.max(cell.z) - 10)

for idx in range(cell.totnsegs):
    ax_morph.plot(cell.x[idx],  cell.z[idx], c='k', lw=1, zorder=-1)
    ax_4s.plot(cell.x[idx], cell.z[idx] + radii[0],
            c='k', lw=0.5)
cell.__del__()

lfp_normalize = np.max([np.max(np.abs(lfp)) for lfp in lfp_dict.values()])
eeg_normalize = np.max([np.max(np.abs(eeg)) for eeg in eeg_dict.values()])

ax_lfp_dict = dict(ylim=[-1350, 50],
                   frameon=False, xticks=[], yticks=[])
ax_eeg_dict = dict(frameon=False, xticks=[], yticks=[],
                   ylim=[-eeg_normalize * 1.05, eeg_normalize/5])

img = None
for i, z_min in enumerate(z_mins):

    ax_grid_lfp = [ax_left + (i + 1) * (ax_w + ax_wspace),
               ax_lfp_hstart, ax_w, ax_h_lfp]
    ax_grid_eeg = [ax_left + (i + 1) * (ax_w + ax_wspace),
               ax_eeg_hstart, ax_w, ax_h_eeg]

    ax_lfp = fig.add_axes(ax_grid_lfp, **ax_lfp_dict)
    ax_eeg = fig.add_axes(ax_grid_eeg, **ax_eeg_dict)
    ax_eeg.text(0.05, 0.01, "EEG", va="bottom")
    ax_lfp.text(0.05, dz / 2, "LFP", va="bottom")
    ax_eeg.text(tstop/2, 0.75, "input above\nz=%d µm" % z_min, ha='center')
    img = plot_laminar_lfp(lfp_dict[z_min], ax_lfp, tvec, lfp_normalize)
    ax_eeg.plot(tvec, eeg_dict[z_min], c='k')

    ax_morph.plot([200 + i * 25, 200 + i * 25], [0, z_min],
                  lw=2, c=depth_clrs[z_min])
    ax_lfp.plot([-1, -1], [dz/2, z_min + dz/2], c=depth_clrs[z_min])

    ax_eeg.plot([tvec[-1] - 1, tvec[-1] - 1], [-0.4, -0.9], c='k', lw=1)
    ax_eeg.text(tvec[-1] - 2, -0.65, "0.5 nV", va="center", ha="right")

    ax_eeg.plot([1, 6], [-0.25, -0.25], c='k', lw=1)
    ax_eeg.text(3.5, -0.35, "5 ms", va="top", ha="center")
    mark_subplots(ax_eeg, "BCD"[i], ypos=2.5, xpos=0.)

mark_subplots(ax_4s, "A", ypos=1.25, xpos=0.05)

cax = fig.add_axes([0.905, 0.05, 0.01, 0.57], frameon=False)
cbar = fig.colorbar(img, cax=cax)
cbar.set_label(r'$V_{\rm e}$ (µV)', labelpad=0)

fig.savefig("fig_eeg_is_simpler.pdf")